In [ ]:
#!pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests

# WebScrapping using BeautifulSoup
BeautifulSoup e Requests são duas bibliotecas comuns em Python para web scraping. Com a biblioteca Requests, você faz uma requisição HTTP para obter o HTML da página a partir de um URL. Com o HTML em mãos, você usa o BeautifulSoup para navegar pela tags e classes do documento e extrai as informações de interesse.

O presente notebook tem por objetivo fazer desenvolver algumas funções e utilizades simples quanto as bibliotecas e por sua vez demonstrar algumas possiiblidades de ações.

# Declaração de funções com uso do BS4
A partir das funções do beautifulsoup(BS4) desenvolvi algumas funções simples que podem auxiliar na busca de textos, links e notícias em uma pesquisa do Google. A ideia é tornar a busca mais dinâmica.

### busca_google()
Para tal, observei que o padrão de adição de palavras na query de busca do Google separa cada palavra da busca na barra por "+", sendo assim a primeira função realiza uma busca no Google a partir das palavras recebidas como parâmetros, caso não receba nenhum parâmetros a função faz a requisição dos termos de busca por um input.

OBS:  
**'User-Agent' no  header do request:** Enviar o user-agent no header faz com que o servidor retorne o mesmo conteúdo que enviaria a um navegador real; sem isso, a resposta pode ser diferente ou bloqueada.

In [ ]:
def busca_google(termos = None):
    soup = None
    # Garante que o retorno do request terá o mesmo conteúdo do navegador
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}

    if termos is None:
        termos = input("Pesquisar no google:")
    else:
        termos = termos.replace(" ", "+")
    
    url = 'https://www.google.com/search?q='+termos
    pagina = requests.get(url, headers = headers)

    if pagina.status_code == 200:
        print("Website found successfully :)")
        print(url) ## Exibindo a url para poder acompanhar os links e tags da página também no navegador
        html = pagina.text
        soup = BeautifulSoup(html, features='html.parser')
        return soup
    else: 
        print("Something went wrong :(")
        return None
    
soup = busca_google("campo receptivo cnn")

### seleciona_atr()
Essa função  recebe como parâmetros um objeto BS4, o seletor HTML e o atributo desejado. Assim, a função percorre elementos HTML do BS4 extraindo os valores selecionados de um atributo específico ou o texto contido neles, com a opção de personalizar o elemento e o atributo desejado.

Como a retirada do texto das tags utiliza uma função distinta para que trechos html não venham juntos se o atributo for "text", o texto de cada elemento é adicionado à lista de retorno. Caso contrário, a função verifica se o atributo existe e, se sim, adiciona o valor correspondente; se o atributo não existir, None é adicionado à lista. 

In [ ]:
def seleciona_atr(soup, selecao = 'a', atr = 'href'):    
    retorno = []
    items = soup.select(selecao)
    for item in items:
        if atr == "text":
            texto = item.get_text()
            retorno.append(item.get_text())

        elif item.has_attr(atr): 
            valor = item.get(atr) # Caso o atributo não existisse o retorno da função seria None
            retorno.append(valor)
        else:
            retorno.append(None)
    return retorno
 
 # ALGUNS EXEMPLOS DE USO:
 
# links encontrados da busca
seleciona_atr(soup,selecao= "div.yuRUbf>div>span>a", atr ="href")

# Links das próximas paginas da busca
seleciona_atr(soup,selecao= "a.fl", atr ="href")

# TEXTOS
seleciona_atr(soup,selecao= "div.VwiC3b>span", atr ="text")

# TÍTULOS
seleciona_atr(soup,selecao= "h3.LC20lb", atr ="text")

## cria_soup()
Apesar de parecer redundante, fiz uma função separada para criar objetos BS4 independentes. O objetivo é que a partir da minha busca google, selecionar os links indexados e tentar fazer a busca do conteúdo dos links. Fazendo assim um aprofundamento do conteúdo das buscas nos sites indexados ao google.  

Para evitar quebras nos códigos eu inclui um _try-catch_, pois alguns dos links encontrados recorrentemente no google são de arquivos PDF, WORD e não apenas páginas web.

In [ ]:
def cria_soup(url = None):
    soup = None
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}

    try:
        pagina = requests.get(url, headers=headers)
        if pagina.status_code == 200:
            html = pagina.text
            soup = BeautifulSoup(html, features='html.parser')
            return soup
        else:
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
# Exemplo de aplicação:
busca_soup = busca_google("campo receptivo cnn")
links_indexados = seleciona_atr(busca_soup,selecao= "div.yuRUbf>div>span>a", atr ="href") # links encontrados da primeira busca
soups_links = []
for link in links_indexados:
    soup_aux = cria_soup(link)
    
    # Verificar se 'soup' é do tipo BeautifulSoup
    if isinstance(soup_aux, BeautifulSoup):
        soups_links.append(soup_aux)

print(f"Soups encontrados: {len(soups_links)}") 
# Tipos dentro da lista
for obj in soups_links:
    print(type(obj))
# Conteúdo
print((soups_links[2].prettify()))

## extrair_texto_limpo()
Visto que não estou utilizando uma estrutura única de seleção de texto a partir de agora, o conteúdo da página ao ser retornado pode conter muito mais textos indesejados e que não fazem sentido para uma análise dos texto de conteúdo das páginas. Sendo assim, esta função tem como objetivo remeover elementos da estrutura de páginas de websites que geralmente não contêm um conteúdo relevante do assunto.  
A função identifica e remove tags HTML que não são relevantes para a análise de conteúdo, como <\script>, <\style>, <\header>, <\footer>, <\nav>, e <\aside>. Esses elementos são comuns em páginas web, mas normalmente não contêm o texto principal que você deseja extrair.

Utilizando soup.get_text(separator=" ") o conteúdo textual das diferentes partes da página são concatenados com um espaço como separador, criando uma continuidade no texto.

Para melhorar ainda mais a legibilidade e a utilidade do texto extraído, a função remove múltiplas quebras de linha e espaços em branco extras. Isso é feito dividindo o texto em uma lista de palavras e, em seguida, juntando essas palavras com um único espaço entre elas.

In [ ]:
def extrair_texto_limpo(soup):
    # Remove elementos desnecessários
    for script in soup(["script", "style", "header", "footer", "nav", "aside"]):
        script.extract()
    
    # Extrair o texto
    texto = soup.get_text(separator=" ")

    # Limpar o texto, removendo múltiplas quebras de linha e espaços extras
    texto_limpo = ' '.join(texto.split())
    
    return texto_limpo


print(extrair_texto_limpo(soups_links[1]))

## remove_stopwords()
Para fazer a análise do conteúdo dos texto de forma sumarizada, sistemática e menos qualitativa farei a remoção das _Stopwords (palavras de pouca relevância)_ do texto utilizando ferramentas da biblioteca `NLTK`. Esta função baixa o conjunto de stopwords  de diversos idiomas, incluindo o português. As stopwords são armazenadas localmente e usadas para filtrar palavras comuns que geralmente não agregam muito significado ao texto.

O texto é divido em palavras e então são removidas as stopwords pela função de tokenização da NLTK. Se a palavra não estiver na lista de stopwords, ela é mantida; caso contrário, é removida.

Essa função pode ser usada como parte de um pipeline de pré-processamento de texto em projetos de NLP, tornando mais fácil e rápido limpar e preparar dados textuais.

OBS:
- Um corpus é uma coleção de textos que podem ser usados para análise de linguagem natural.

In [ ]:
# Preparação do ambiente:
import nltk
from nltk.corpus import stopwords # É um dos conjuntos de dados inclusos em corpus incluindo diferentes idiomas.
from nltk.tokenize import word_tokenize # Contém funções para dividir o texto em unidades menores, como palavras ou frases.

#nltk.download('stopwords') # Garante que as listas de stopwords dos idiomas estejam disponíveis localmente no ambiente de desenvolvimento
#nltk.download('punkt') # A word_tokenize() para dividir uma string de texto em palavras, o NLTK utiliza o modelo punkt para identificar corretamente os limites das palavras
#nltk.download('punkt_tab')
def remove_stopwords(texto, idioma = "portuguese"):
    # Obter a lista de stopwords para o idioma especificado
    stop_words = set(stopwords.words(idioma))
    # Tokenizar o texto
    palavras = word_tokenize(texto)
    
    # Filtrar as palavras que não são stopwords
    palavras_filtradas = []
    for palavra in palavras:
        if palavra.lower() not in stop_words and palavra.isalpha():
            palavras_filtradas.append(palavra.lower())
    
    # Juntar as palavras filtradas de volta em uma string
    texto_limpo = ' '.join(palavras_filtradas)
    
    return texto_limpo

# Exemplo de uso
texto_exemplo = "Este é um exemplo de frase com algumas palavras irrelevantes."
texto_processado = remove_stopwords(texto_exemplo)
print(texto_processado)

## verifica_texto_legivel()
A função determina se o texto extraído de um documento ou página web é, de fato, "legível" e não apenas uma sequência de caracteres sem sentido, o que pode ocorrer caso o contéudo da página exista mas esteja em um formato de mídia ou arquivo binario. A ideia é verificar se a maior parte do texto consiste em caracteres alfanuméricos com letras, números e espaços.


In [1]:
def verificar_texto_legivel(texto):
    # Conta o número de caracteres "legíveis", ou seja, letras, números e espaços
    caracteres_legiveis = sum(c.isalnum() or c.isspace() for c in texto)
    
    # Calcula a proporção de caracteres legíveis em relação ao comprimento total do texto
    proporcao_legivel = caracteres_legiveis / len(texto)
    
    # Se mais de 70% do texto é legível, consideramos que é texto válido
    return proporcao_legivel > 0.7

texto_valido = "Este é um exemplo de texto legível e normal."
texto_invalido = "\x00\x01\x02\x03\x04\x05\x06\x07\x08\x09"

print(verificar_texto_legivel(texto_valido))   # Deve retornar True
print(verificar_texto_legivel(texto_invalido)) # Deve retornar False


True
False
